In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import datetime
import time

In [3]:
data_list = [0,0,0,0]
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service)
url = "https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month"
browser.get(url)
#スクレイピング開始時間取得
start_time = time.time()
#画像表示まで待機
wait = WebDriverWait(browser, 5)
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#PAGE_TOP > div > div > div > a > img')))
#HTML取得
source = browser.page_source
kabu_data = ""

In [4]:
def extract_stock_data(source_data):
    soup = BeautifulSoup(source_data,"html.parser")
    kabu_data = soup.find('tr')
    count = 0
    for i in (kabu_data):
        kabu_value = i.text
        if count == 0:
            date=datetime.datetime.strptime(kabu_value,"%Y/%m/%d")
            kabu_value=date.strftime("%Y-%m-%d")
            if kabu_value == data_list[0]:
                break
            data_list[0] = kabu_value
        if 1 <= count <= 3:
            data_list[count] = kabu_value
        if count == 4:
            print(data_list)
        count = count + 1

In [5]:
graph = browser.find_element(By.CLASS_NAME,"m-trend_chart_area")
actions = ActionChains(browser)
actions.move_to_element(graph)
actions.perform()
#グラフの右橋にマウスを移動
actions.move_by_offset(- (646) // 2,0)
actions.perform()

In [6]:
for i in range(0,10):
    actions.move_by_offset(i,0)
    actions.perform()
    source = browser.page_source
    extract_stock_data(source)

['2023-04-28', '始値: 28,705.91', '高値: 28,879.24', '安値: 28,499.51']
['2023-05-01', '始値: 29,058.05', '高値: 29,145.89', '安値: 29,016.83']
['2023-05-02', '始値: 29,278.8', '高値: 29,278.8', '安値: 29,083.13']
['2023-05-09', '始値: 29,020.63', '高値: 29,262.36', '安値: 29,020.63']
['2023-05-10', '始値: 29,189.41', '高値: 29,195.16', '安値: 29,070.11']
['2023-05-12', '始値: 29,199.3', '高値: 29,426.06', '安値: 29,141.52']
['2023-05-15', '始値: 29,547.04', '高値: 29,629.47', '安値: 29,475.97']


In [7]:
#終了作業
browser.close()
print("スクレイピングにかかった時間")
print(time.time() - start_time)

スクレイピングにかかった時間
21.594072818756104
